### Trade A Series Of Markets

This notbook induces agents developed to trade a market, using the ML model developed in the training notebooks in the notebook folder.

The steps are:

1. Get a list of markets for the day
1. Activate all markets within the trading window (20m), using python threads to run as a background process
1. Sleep the script until we are ready to invoke another market

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import betfairlightweight
import datetime
from datetime import datetime as dt
import flumine
from flumine.resources import recorder
from flumine.storage import storageengine
import json
import numpy as np
import pandas as pd
import time
from agent_bsp import Agent
from threading import Thread

#### Get market list

Get the markets from betfair through the API and then initiate the flumine stream.

In [ ]:
HOURS = 6
event_id = 7 # Horse Racing ID

In [ ]:
with open('secrets/config.json', 'r') as fp:
    config = json.load(fp)

trading = betfairlightweight.APIClient(**config)
trading.login()

In [ ]:
# TODO: Move to utils
def get_markets(event, trading):
    market_catalogue_filter = betfairlightweight.filters.market_filter(event_ids=event)

    market_catalogues = trading.betting.list_market_catalogue(
        filter=market_catalogue_filter,
        max_results='100',
        sort='FIRST_TO_START'
    )

    # Create a DataFrame for each market catalogue
    market_types_df = pd.DataFrame({
        'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
        'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
        'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
    })
    
    market_types_df['Event ID'] = event
    
    return market_types_df

In [ ]:
# Define a market filter
event_filter = betfairlightweight.filters.market_filter(
    event_type_ids=[event_id],
    market_countries=['AU'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=HOURS)).strftime("%Y-%m-%dT%TZ")
    }
)

# Get a list of all thoroughbred events as objects
events = trading.betting.list_events(
    filter=event_filter
)

# Create a DataFrame with all the events by iterating over each event object
event_df = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in events],
    'Event ID': [event_object.event.id for event_object in events],
    'Event Venue': [event_object.event.venue for event_object in events],
    'Country Code': [event_object.event.country_code for event_object in events],
    'Time Zone': [event_object.event.time_zone for event_object in events],
    'Open Date': [event_object.event.open_date for event_object in events],
    'Market Count': [event_object.market_count for event_object in events]
})

In [ ]:
# event_df = event_df[(event_df['Event Name'].str.contains('Rand'))|(event_df['Event Name'].str.contains('Flem'))]
event_df

In [ ]:
track_dict = event_df[['Event Name', 'Event Venue']]
track_dict['Event Name'] = track_dict['Event Name'].str.split(' ').apply(lambda x: x[0])
track_dict = dict(track_dict.set_index('Event Venue').to_records())

In [ ]:
market_catalogue_filter = betfairlightweight.filters.market_filter(event_ids=event_df['Event ID'].values.tolist(), market_type_codes=['WIN'])

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    max_results='500',
    sort='FIRST_TO_START'
)

# Create a DataFrame for each market catalogue
market_df = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
})

In [ ]:
market_filter = {"marketIds": market_df['Market ID'].values.tolist()}
storage_engine = storageengine.Local('./data')
rcd =  recorder.MarketRecorder(storage_engine, market_filter=market_filter)

In [ ]:
flumine_object = flumine.flumine.Flumine(
    recorder=rcd,
    settings={'certificate_login': True, 'betfairlightweight': config}
)

In [ ]:
flumine_object.start()

In [ ]:
while flumine_object.status != 'running':
    time.sleep(3)
time.sleep(10)

In [ ]:
rcd.stream_id

In [ ]:
mrkt_times = []
for market in market_df['Market ID']:
    try:
        with open(f'./data/{rcd.stream_id}/{market}') as f:
            m_time = json.loads(f.readline())['mc'][0]['marketDefinition']['marketTime']
        mrkt_times.append(m_time)
    except:
        mrkt_times.append(None)

Below is the final market trading df, that has the markets we will trade

In [ ]:
market_df['startTime'] = mrkt_times
market_df['startTime'] = pd.to_datetime(market_df.startTime)
market_df['tradeTime'] = market_df.startTime - np.timedelta64(20, 'm')
market_df['auTime'] = market_df['startTime'].dt.tz_convert('Australia/ACT')
# market_df

In [ ]:
trading.logout()

#### Trading a single funciton
The main trading function, used for each market

In [ ]:
def trade_main(market, rcd, start, end, model='./models/bsp/XRT_1_AutoML_20200109_105830/XRT_1_AutoML_20200109_105830.zip'):  
    try:
        end = np.datetime64(end)
        if np.datetime64('now') > end:
            return None

        with open('secrets/config.json', 'r') as fp:
            config = json.load(fp)

        trading = betfairlightweight.APIClient(**config)
        trading.login()

        bot = Agent(market, trading, rcd, model)

        print(f'Start Trading on {market}\n')
        while np.datetime64('now') < end:
            bot.poll_directory()
            if not bot.in_market:
                bot.assess_bsp(verbose=False)
            bot.manage_trade()
        bot.cashout()

        trading.logout()
        print(market, f'{market} END END END\n')
    except Exception as e:
        print(e)
    

#### Trading loop
Primary loop that starts trading markets at appropraite times

In [ ]:
trading_threads = {}
for row in market_df.itertuples(index=False):
    race, market, trade_vol, end, start, local = row
    if np.datetime64('now') < start.tz_localize(None):
        td = start.tz_localize(None) - np.datetime64('now')
        print(f'Sleeping {td.total_seconds()/60} Minutes')
        time.sleep(td.total_seconds())
    trading_threads[market] = Thread(target=trade_main, args=(market, rcd, start, end,))
    trading_threads[market].start()
    

In [ ]:
flumine_object.stop()